## Моделирование методом Монте-Карло ##

In [1]:
from numpy import *
from numpy.random import random
sq = square
N = 100000
rnd = lambda: random(N)

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

In [3]:
#%matplotlib notebook 

In [4]:
def gen_k(K):
    kx = K *(2 * rnd() - 1)
    l = sqrt(sq(K) - sq(kx))
    kz = l * (2 * rnd() - 1)
    ky = sqrt(sq(K) - sq(kx) - sq(kz)) * np.random.choice([-1,1], N)
    return kx, ky, kz

In [5]:
def dfdy(k, M, m, R):
    a = sqrt(1 + sq(R) / sq(M))
    kx, ky, kz = k
    
    E1 = a * M / 2 + kx * R / M
    p1x = R / 2 + a * kx
    p1y = ky
    p1z = kz
    
    E2 = a * M / 2 - kx * R / M
    p2x = R / 2 - a * kx
    p2y = -ky
    p2z = -kz
    
    #tg1 = p1y / p1x
    #tg2 = p2y / p2x
    #tgdf = (tg1 - tg2) / (1 - tg1 * tg2)
    #df = arctan(tgdf)
    
    f1 = arctan2(p1y, p1x)
    f2 = arctan2(p2y, p2x)
    df = f1 - f2
    
    y1 = 0.5 * log((E1 + p1z)/ (E1 - p1z))
    y2 = 0.5 * log((E2 + p2z)/ (E2 - p2z))
    dy = y1 - y2
    
    return dy, df

In [6]:
def string(dphi):
    y = cos(dphi) / 2
    c = sqrt(1 - sq(y))
    return (3 / (8 * pi)) * (c - y * arccos(y)) / (c * c * c)

def delta(Y):
    res = []
    for y in Y:
        s = array([0 if abs(d) > 0.1 else 1 for d in (y - 1)])
        s += array([0 if abs(d) > 0.1 else 1 for d in (y + 1)])
        #s = 1 - cos(pi * y)
        res.append(s)
    return np.array(res)

In [7]:
M = 0.775
m = 0.140
R = 1.3000
K = sqrt(sq(M / 2) - sq(m))
sigma_x = 0.5
sigma_y = 0.4
d = 1000
dx = 0.1
dy = pi / 20
spi = sqrt(2 * pi)

In [12]:
def show(R, sigma_x=0.5, sigma_y=0.4, d=1000):
    X = np.arange(-2, 2.01, dx)
    Y = np.arange(-pi , pi + 0.01, dy)
    Z, X, Y = histogram2d(*dfdy(gen_k(K), M, m, R), bins=(X, Y))
    X = mean([X[1:], X[:-1]], axis=0)
    Y = mean([Y[1:], Y[:-1]], axis=0)
    Z = Z.T
    X, Y = np.meshgrid(X, Y)
    #X.shape, Y.shape, Z.shape, mesh[1].shape

    W = delta(X) * string(Y)
    Z = Z + d * W

    Z_ = np.zeros(Z.shape)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            k_range = int(3 * sigma_x / dx)
            for k in range(-k_range, k_range + 1):
                if (j + k >= 0 and (j + k < Z.shape[1])):
                    Z_[i, j] += Z[i ,j + k] * exp(-sq(dy * k) / sq(sigma_y)) / sigma_y / spi

    Z__ = np.zeros(Z.shape)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            k_range = int(3 * sigma_y / dy)
            for k in range(-k_range, k_range + 1):
                if (i + k >= 0 and (i + k < Z.shape[0])):
                    Z__[i, j] += Z_[i + k ,j ] * exp(-sq(dx * k) / sq(sigma_x)) / sigma_x / spi

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    #ax = fig.gca(projection='3d', xlabel=, ylabel="$\Delta \phi$", title="R = " + str(R) + " GeV")
    surf = ax.plot_surface(X, Y, Z__ / sum(Z), cmap=cm.coolwarm, linewidth=0, antialiased=False)
    #ax.set_zlim3d(0.0, 1.0)
    x_label = ax.set_xlabel("$\Delta y$")
    y_label = ax.set_ylabel("$\Delta \phi$")
    x_label.set_fontsize(14)
    y_label.set_fontsize(14)
    #ax.set_zlabel(r"")
    ax.view_init(45, -45)
    plt.savefig("R" + str(R) + ".png")


In [13]:
show(2,sigma_x=0.2, sigma_y=0.2, d=0)

In [14]:
show(3,sigma_x=0.2, sigma_y=0.2, d=0)

In [15]:
show(2.5,sigma_x=0.2, sigma_y=0.2, d=0)

In [16]:
show(1,sigma_x=0.2, sigma_y=0.2, d=0)

In [17]:
show(1.5,sigma_x=0.2, sigma_y=0.2, d=0)

In [18]:
show(5,sigma_x=0.2, sigma_y=0.2, d=0)

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact
from bokeh.models import ColumnDataSource

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

In [ ]:
def present(f_gen, f_calc, title = "", a = 7, b = 4, l = 0.1, r = 1):
    p = figure(title=title, plot_height=400, plot_width=900)
    
    hist, edges = np.histogram(f_gen(a, b), density=True,normed=True, bins=100)
    gen = p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="#036564", line_color="#033649", legend="Monte-Carlo")
    
    m = 1000 / (r - l)
    x = np.linspace(l, r, 1000)
    #calc = p.line(x=x, y= m * f_calc(x, a, b), line_color="#D95B43", line_width=8, alpha=0.7, legend="Calculated")

    def update(a = 7, b = 4):
        hist, edges = np.histogram(f_gen(a, b), density=True, normed=True, bins=100)
        gen.data_source.data['top'] = hist
        #calc.data_source.data['y'] = m * f_calc(x, a, b)
        
        push_notebook()

    show(p, notebook_handle=True)
    interact(update, a=(0.1,10, 0.1), b=(0.1, 10, 0.1))

In [ ]:
def show2(R, sigma_x=0.5, sigma_y=0.4, d=1000):
    X = np.arange(-2, 2.01, dx)
    Y = np.arange(-pi , 2 * pi + 0.01, dy)
    Z, X, Y = histogram2d(*dfdy(gen_k(K), M, m, R), bins=(X, Y))
    X = mean([X[1:], X[:-1]], axis=0)
    Y = mean([Y[1:], Y[:-1]], axis=0)
    Z = Z.T
    X, Y = np.meshgrid(X, Y)
    #X.shape, Y.shape, Z.shape, mesh[1].shape

    W = delta(X) * string(Y)
    Z = Z + d * W

    Z_ = np.zeros(Z.shape)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            k_range = int(3 * sigma_x / dx)
            for k in range(-k_range, k_range + 1):
                if (j + k >= 0 and (j + k < Z.shape[1])):
                    Z_[i, j] += Z[i ,j + k] * exp(-sq(dy * k) / sq(sigma_y)) / sigma_y / spi

    Z__ = np.zeros(Z.shape)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            k_range = int(3 * sigma_y / dy)
            for k in range(-k_range, k_range + 1):
                if (i + k >= 0 and (i + k < Z.shape[0])):
                    Z__[i, j] += Z_[i + k ,j ] * exp(-sq(dx * k) / sq(sigma_x)) / sigma_x / spi

    fig = figure(plot_height=400, plot_width=900)
    ax = fig.gca(projection='3d', xlabel="$\Delta y$", ylabel="$\Delta \phi$", title="R = " + str(R) + " GeV")
    surf = ax.plot_surface(X, Y, Z__ / sum(Z), cmap=cm.coolwarm, linewidth=0, antialiased=False)
    show(fig)

In [ ]:
show2(1)